<a href="https://colab.research.google.com/github/Harshit-kumar-msit/GrameenCare/blob/main/med_chat_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_groq langchain langchain_core langchain_community sentence_transformers chromadb pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 88.0 MB/s eta 0:0

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 96.8 MB/s eta 0:00:00


In [ ]:
from langchain_groq import ChatGroq


In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import gradio as gr

In [ ]:
def initialize_llm():
  llm=ChatGroq(
    temperature=0,
    groq_api_key="*************************************",
    model_name="llama-3.3-70b-versatile"
  )
  return llm
def create_vectordb():
  loader=DirectoryLoader("/content/data/",glob="*.pdf",loader_cls=PyPDFLoader)
  documents=loader.load()
  text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
  texts=text_splitter.split_documents(documents)
  embeddings=HuggingFaceBgeEmbeddings(model_name="sentence-transformer/all-MiniLM-L6-v2")
  vectordb=Chroma.from_documents(texts,embeddings,persist_directory="./chromadb")
  vectordb.persist()
  return vectordb
def setup_qa_chain(vectordb,llm):
  retriever=vectordb.as_retriever()
  prompt_templates="""You are a compassionate medical chatbot. Respond thoughtfully to the following question:
  {context}
  User:{question}
  Chatbot: """
  prompt=PromptTemplate(template=prompt_templates,input_variables=["context","question"])
  qa_chain=RetrievalQA.from_chain_type(
      llm=llm,
      chain_type="stuff",
      retriever=retriever,
      chain_type_kwargs={"prompt":prompt}
  )
  return qa_chain


def chatbot_response(user_input,history=[]):
  if not user_input.strip():
    return {"text":"Please provide a Valid input","history":history}
  db_path="/content/chroma_db"
  if not os.path.exists(db_path):
    vectordb=create_vectordb()
  else:
    embeddings = HuggingFaceBgeEmbeddings(model_name="pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")
    vectordb=Chroma(persist_directory=db_path,embedding_function=embeddings)
  llm=initialize_llm()
  qa_chain=setup_qa_chain(vectordb,llm)
  response=qa_chain(user_input)
  text_response = response.get('result', 'Sorry, I couldn\'t find an answer.')
  history.append((user_input,text_response))
  return {"text": text_response, "history": history}


with gr.Blocks() as app:
  chatbot=gr.ChatInterface(fn=chatbot_response,title="Mental Health Chatbot")

app.launch(debug=True)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://dc3ff30a4fe41a6a8d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://dc3ff30a4fe41a6a8d.gradio.live
